###url 가져오기

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import re
import time
import pandas as pd

In [10]:
chrome_path = "C:\\Users\\wmk51\\python-project\\chromedriver.exe"
options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(options=options)
url = "https://www.musinsa.com/search/musinsa/goods?q=%EB%8B%88%ED%8A%B8&list_kind=small&sortCode=pop&sub_sort=&page=1&display_cnt=0&saleGoods=&includeSoldOut=&setupGoods=&popular=&category1DepthCode=&category2DepthCodes=&category3DepthCodes=&selectedFilters=&category1DepthName=&category2DepthName=&brandIds=&price=&colorCodes=&contentType=&styleTypes=&includeKeywords=&excludeKeywords=&originalYn=N&tags=&campaignId=&serviceType=&eventType=&type=&season=&measure=&openFilterLayout=N&selectedOrderMeasure=&shoeSizeOption=&d_cat_cd=&attribute=&plusDeliveryYn="
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()
WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="footerCommonPc"]/div[1]/button[3]'))).click()##여성 버튼
time.sleep(1)

In [11]:
html_url = driver.page_source
soup_url = BeautifulSoup(html_url,"html.parser")
time.sleep(1)


product_url_list=[]
url_list = soup_url.find_all("a" , attrs={"class" : "img-block"})[:30]

for item in  url_list:
    product_url_list.append(item["href"])

In [15]:
from collections import defaultdict

In [12]:
num = 0
prod_avg_dict={}
for product_url in product_url_list[:30]:
    
    driver.get(product_url)
    driver.set_window_size(1920,1080)
    driver.maximize_window()
        
        
    ##최신 순으로 댓글 정렬    
    html_url = driver.page_source
    soup_url = BeautifulSoup(html_url,"html.parser")
    time.sleep(2)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "reviewSelectSort"))).click()
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//option[@value="new"]'))
    ).click()
    
    num+=1
    html_product = driver.page_source
    soup_product = BeautifulSoup(html_product,"html.parser")
    time.sleep(1)
    #상품 이름
    prod_name = soup_product.find("h3",attrs={"class":"product-detail__sc-1klhlce-3"}).get_text()
    a = True
    i=0
    date_list=[]
    rating_list=[]
    reviewer_id_list = []
    tag_list = []
    print(num,"번쨰 상품")
    
    
    i=0
    while a:
    ## i를 5로 나눠서 나머지가 0이면 next버튼을 눌러준다
    
        i+=1
        if( i % 5 == 0):
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".fa.fa-angle-right.paging-btn.btn.next"))).click()
            time.sleep(2)
            
        else:
            try:
                WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, f'//a[text()={i}]'))).click()
                time.sleep(2)
            except:
                break
        
        ##페이지 전환 후 다시 페이지 소스를 가져온다
        html_product = driver.page_source
        soup_product = BeautifulSoup(html_product,"html.parser")
        
        
        
        
        
        
        
        ### 별점 가져오기
        # 5개는 100, 4개는 80, 3개는 60, 2개는 40, 1개는 20
        stars = soup_product.find_all("span", attrs={"class":"review-list__rating__active"})

        for star in stars:
            
            rating_list.append(re.sub(r'\D', '', star['style'])) 
                
        # 리스트 안에 값을 int로 변환
        rating_list = [int(x) for x in rating_list]
        ##남성 별점
        avg_rating = sum(rating_list) / len(rating_list)
        
        

        ### 작성자 가져오기
        
        
        # reviewers= soup_product.find_all("p",attrs={"class" : "review-profile__name"})
        
        # for reviewer in reviewers:
            
        #     reviewer_id_list.append(reviewer.get_text())
        #작성일 가져오기
        #break문을 사용해서 확실히 끊어야 한다.
        
        dates = list(soup_product.find_all("p", attrs={"class":"review-profile__date"}))

        
        for date in dates:
            date = date.get_text()
            if '.' in date:
                a=False
                #마지막 페이지 date 가져오는 거 그만
                break
            
            date_list.append(date)
        # print("date 길이 : ", len(date_list))  
            
        
        
        # print("review_id : ",len(reviewer_id_list),"rating : ",len(rating_list),"review_date : ",len(date_list))
    
    
    
        tags = soup_product.find_all("li",attrs={"class" : "review-evaluation--type3__item"})
        my_dict={}
        for tag in tags:
            
            my_dict={
                tag.get_text().split(" ")[0]:tag.get_text().split(" ")[1]
                
            }
            # 기간 내에 있는 모든 테그들을 담은 리스트
            tag_list.append(my_dict)
    # prod_avg_dict ={
    #     prod_name : avg_rating
    # }
    prod_avg_dict[prod_name] =avg_rating
    print("평균 딕셔너리 : ",prod_avg_dict)
    
    # ##상품 별 키워드를 가져와서 키워드 카테고리별 최고 빈도 키워드를 가져온다
    # # 빈도수를 저장할 딕셔너리
    # frequency = defaultdict(lambda: defaultdict(int))
    # # 각 키별로 값을 세기
    # for dictionary in tag_list:
    #     for key, value in dictionary.items():
    #         # print(key,value)
    #         # ex key가 사이즈면 해당 value값의 빈도가 올라감
    #         frequency[key][value] += 1

    # keyword_dict = {}
    # # 각 키별로 빈도가 가장 높은 값을 출력
    # for key, values in frequency.items():
    #     # print("key :" ,key,"value : ",value)
    #     max_value_key = max(values, key=values.get)
    #     #key = 사이즈 : 봉통이에요 - value["보통이에요"]
    #     keyword_dict[key] = [max_value_key]
        
    # product_tag_rank_df = pd.DataFrame(keyword_dict)
    # product_tag_rank_df.to_csv(f'./men_musinsa_hashtag/{prod_name}.csv' , index=False,encoding='utf-8-sig')
        
      
    
    #상품 별 csv 파일 만들기        
    # data_dict = {
    #                 "review_id" : reviewer_id_list[:len(date_list)],
    #                 "rating" : rating_list[:len(date_list)],
    #                 "review_date" : date_list
    #             }  
    # product_df = pd.DataFrame(data_dict)
    # product_df.to_csv(f'./men_musinsa_price_df/{prod_name}_tag.csv', index=False,encoding='utf-8-sig')   
        
        
        
# men_avg_price_df = pd.DataFrame(prod_avg_dict)
        
    
    


1 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0}
2 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0, '[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트': 96.4}
3 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0, '[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트': 96.4, 'HERITAGE DAN CABLE KNIT HOODIE ZIP-UP IVORY': 100.0}
4 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0, '[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트': 96.4, 'HERITAGE DAN CABLE KNIT HOODIE ZIP-UP IVORY': 100.0, '2WAY 헤비웨이트 케이블 니트 후드 집업_오트밀': 96.28571428571429}
5 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0, '[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트': 96.4, 'HERITAGE DAN CABLE KNIT HOODIE ZIP-UP IVORY': 100.0, '2WAY 헤비웨이트 케이블 니트 후드 집업_오트밀': 96.28571428571429, '컬러 블록 래글런 니트_버건디_KN124': 98.0}
6 번쨰 상품
평균 딕셔너리 :  {'HOFE 니트 후드 후드집업 티 그레이': 88.0, '[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트': 96.4, 'HERITAGE DAN CABLE KNIT HOODIE ZIP-UP IVORY': 100.0, '2WAY 헤비웨이트 케이블 니트 후드 집업_오트밀': 96.28571428571429, '컬러 블록 래글런 니트_버건디_KN124': 98.0, '[2컬러] 스트라이프 언발란스 오프숄더 스웨터': 

In [13]:
prod_avg_dict.keys()
data_dict = {
    'Item' : prod_avg_dict.keys(),
    "평점" : (prod_avg_dict.values())
}

In [16]:
df_rating_women = pd.DataFrame(data_dict)
df_rating_women.to_csv("women_rating.csv", index=False,encoding='utf-8-sig') 

In [17]:
df_rating_women

,Item,평점
0,HOFE 니트 후드 후드집업 티 그레이,88.000000
1,[2-PACK] (하프집업 + 크루넥 선택) 코튼 워셔블 하찌 니트,96.400000
2,HERITAGE DAN CABLE KNIT HOODIE ZIP-UP IVORY,100.000000
3,2WAY 헤비웨이트 케이블 니트 후드 집업_오트밀,96.285714
4,컬러 블록 래글런 니트_버건디_KN124,98.000000
5,[2컬러] 스트라이프 언발란스 오프숄더 스웨터,94.000000
6,데일리 라운드 니트 - 8 COLOR,98.272727
7,WOMENS HERITAGE DAN CABLE KNIT HOODIE ZIP-UP I...,100.000000
8,카라 케이블 니트 집업 블랙,99.000000
9,UB 베이직 니트 후드 집업_그레이_KN122,100.000000


In [17]:

print(len(reviewer_id_list),len(rating_list))


0 10


In [18]:
reviewer_id_list = reviewer_id_list[:len(date_list)]
rating_list = rating_list[:len(date_list)]

print("작성자 : ",len(reviewer_id_list))
print("별점 : ",len(rating_list))
print("작성일 : ",len(date_list))

작성자 :  0
별점 :  8
작성일 :  8


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
data_dict = {

    "review_id" : reviewer_id_list,
    "rating" : rating_list,
    "review_date" : date_list
    
}

In [21]:
product_df = pd.DataFrame(data_dict)
product_df

ValueError: All arrays must be of the same length

In [ ]:
####여기서 부터는 태그 데이터  피처링

In [ ]:
tag_list

[{'사이즈': '보통이에요'},
 {'밝기': '보통이에요'},
 {'색감': '보통이에요'},
 {'두께감': '보통이에요'},
 {'촉감': '부드러워요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '커요'},
 {'밝기': '어두워요'},
 {'색감': '보통이에요'},
 {'두께감': '보통이에요'},
 {'촉감': '부드러워요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '보통이에요'},
 {'색감': '선명해요'},
 {'두께감': '보통이에요'},
 {'촉감': '부드러워요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '보통이에요'},
 {'색감': '보통이에요'},
 {'두께감': '두꺼워요'},
 {'촉감': '거칠어요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '보통이에요'},
 {'색감': '선명해요'},
 {'두께감': '두꺼워요'},
 {'촉감': '평범해요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '어두워요'},
 {'색감': '보통이에요'},
 {'두께감': '두꺼워요'},
 {'촉감': '평범해요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '밝아요'},
 {'색감': '선명해요'},
 {'두께감': '두꺼워요'},
 {'촉감': '부드러워요'},
 {'배송': '빨라요'},
 {'포장': '꼼꼼해요'},
 {'사이즈': '보통이에요'},
 {'밝기': '보통이에요'},
 {'색감': '선명해요'},
 {'두께감': '두꺼워요'},
 {'촉감': '평범해요'},
 {'배송': '빨라요'},
 {'포장': '아쉬워요'},
 {'사이즈': '보통이에요'},
 

In [ ]:
data_dict = {
    '사이즈': ['보통이에요', '커요', '보통이에요', '보통이에요', '보통이에요'],
    '밝기': ['보통이에요', '보통이에요', '보통이에요', '밝아요'],
    '색감': ['선명해요', '선명해요', '보통이에요', '선명해요'],
    '두께감': ['보통이에요', '얇아요', '보통이에요', '보통이에요'],
    '촉감': ['부드러워요', '부드러워요', '평범해요'],
    '배송': ['빨라요', '빨라요', '빨라요'],
    '포장': ['꼼꼼해요', '꼼꼼해요', '꼼꼼해요']
}


In [ ]:
from collections import defaultdict

# 빈도수를 저장할 딕셔너리
frequency = defaultdict(lambda: defaultdict(int))

data = [{'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '보통이에요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '보통이에요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '커요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '얇아요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '보통이에요'}, {'두께감': '보통이에요'}]

# 각 키별로 값을 세기
for dictionary in tag_list:
    for key, value in dictionary.items():
        # print(key,value)
        # ex key가 사이즈면 해당 value값의 빈도가 올라감
        frequency[key][value] += 1
print(frequency)
key_list=[]
# 각 키별로 빈도가 가장 높은 값을 출력
for key, values in frequency.items():
    # print("key :" ,key,"value : ",value)
    max_value = max(values, key=values.get)
    #key = 사이즈 : 봉통이에요 - value["보통이에요"]
    print(f"{key}: {max_value} - {values[max_value]}번")
        

defaultdict(<function <lambda> at 0x0000022CF1DDAC00>, {'사이즈': defaultdict(<class 'int'>, {'보통이에요': 121, '커요': 24, '작아요': 5}), '밝기': defaultdict(<class 'int'>, {'보통이에요': 124, '어두워요': 23, '밝아요': 3}), '색감': defaultdict(<class 'int'>, {'보통이에요': 106, '선명해요': 43, '흐려요': 1}), '두께감': defaultdict(<class 'int'>, {'보통이에요': 77, '두꺼워요': 71, '얇아요': 2}), '촉감': defaultdict(<class 'int'>, {'부드러워요': 57, '거칠어요': 3, '평범해요': 90}), '배송': defaultdict(<class 'int'>, {'빨라요': 135, '아쉬워요': 15}), '포장': defaultdict(<class 'int'>, {'꼼꼼해요': 147, '아쉬워요': 3})})
사이즈: 보통이에요 - 121번
밝기: 보통이에요 - 124번
색감: 보통이에요 - 106번
두께감: 보통이에요 - 77번
촉감: 평범해요 - 90번
배송: 빨라요 - 135번
포장: 꼼꼼해요 - 147번
